<a href="https://colab.research.google.com/github/anderfrago/TrasladarNotasAlumnos/blob/main/2021_TrasladarNotasAlumnos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

#@title # Trasladar notas a los alumnos
#@markdown El siguente Notebook permite trasladar las líneas de un excel al nombre de alumno establecido en la primera columna y compartirlo con el correo de la segunda columna.
#@markdown - El listado de alumnos comienza a partir de la línea X.
#@markdown - Primera columna nombre del alumno, la cabecera (X primeras líneas) y la línea actual se copia a un excel dentro de una carpeta con este nombre
#@markdown - Segunda columna correo del alumno para compartir la carpeta.

#@markdown ## Conexión a GDrive
#@markdown > Debes acceder al enlace que aparece en la ejecución y copiar la clave en la casilla de introducción.
#@markdown ### 1. Montar la unidad de Drive
#@markdown Agregamos a unidad de GDrive al ordenador en la nube de GColab
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#@markdown ### 2. Obteniendo acceso al SDK de Google Drive
#@markdown Las herramientas de desarrollo de Google Drive nos va a permitir crear nuevos elementos en GDrive y compartir estos elementos con el alumnado.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [ ]:
#@markdown ## Ubicación en GDrive
#@markdown Introduce la ubicación donde se encuentra el EXCEL de notas trasladar a los alumnos.
#@markdown *En caso de especificar la ruta completa el proceso será maś rápido*


#@markdown ### Introduce la ruta al directorio
ruta_directorio = "/4Vientos/2021/Notak/1ASIR_ISO" #@param {type:"string"} 
path =  "/content/drive/MyDrive"+ruta_directorio
#@markdown ### Introduce nombre del archivo EXCEL:
nombre_excel_notas = "2122_ISO_NOTAK" #@param {type:"string"} 
filenombre_excel_notas = nombre_excel_notas+'.xlsx'

#@markdown ## Cabecera:
#@markdown ### Introduce el nombre del archivo plantilla:
plantilla_cabecera =  "2122_TMP_ISO_NOTAK"#@param {type:"string"} 
#@markdown ### Introduce el número de líneas de cabecera:
numero_cabecera =  6#@param {type:"number"} 
# numero_cabecera -= 1

listed = drive.ListFile({'q': "title contains '"+filenombre_excel_notas+"'"}).GetList()
try: 
  # Last modified file
  file = listed[0]
  #print(file['id'])
  file_id = file['id']
  print("Archivo de notas localizado exitosamente!")
  '''print(file_id)
  for file in listed:
    file_id = file['id']
    if(file_id) :
        print(file)
        print("Archivo de notas localizado exitosamente!")
  '''

  downloaded = drive.CreateFile({'id': file_id})
  downloaded.GetContentFile(filenombre_excel_notas)
except:
  print("no se ha localizado el archivo de notas "+ filenombre_excel_notas)


In [ ]:

#@markdown ## Copiar notas de manera individual para los alumnos
#@markdown A continuación se realiza el proceso de trasladar cada una de las líneas con notas de alumnos a una carpeta con el nombre del alumno.
#@markdown En la carpeta se genera un excel y se copian las líneas de cabecera junto la nota del alumno

!pip install openpyxl > /dev/null
import openpyxl
import pandas as pd
import time
from shutil import copyfile
from openpyxl import load_workbook

# Read Excel translate to temporary csv
os.chdir('/content')
read_file  = pd.read_excel(filenombre_excel_notas)
read_file.to_csv (r'tmp.csv', index = None, header=None)
df = pd.read_csv('tmp.csv')
os.remove('tmp.csv')
wb_obj = openpyxl.load_workbook(filenombre_excel_notas)
ws_obj = wb_obj.worksheets[0]
###
## Process of tranlating notes to students folder
###
print("Comineza el proceso de trasladar las notas al alumnado")
for index, row in df.iloc[numero_cabecera-2:].iterrows():
  # Get name to create the folder and contact mail
  name = row.values[0]  
  email = row.values[1]
  # Creation of students folder
  print("\t - Trasladando notas al alumn@  "+name)
  try: 
    os.chdir(path)
    if not os.path.isdir(name) :
        print("\t\t Primera vez que se trasladan las notas, creación del directorio del alumn@")
        os.mkdir(name)
        time.sleep(1)

    os.chdir(name)
    # Transformar from numpy.ndarray to excel values
    row_list = row.values.tolist()

    file_name = name+"__"+filenombre_excel_notas
    ## COPY TEMPLATE
    template="..//"+plantilla_cabecera+".xlsx"
    try :
      copyfile(template, file_name)
      time.sleep(1)
      ## COPY TEMPLATE
      # Creation of excel
      wb = load_workbook(file_name)
      # Select First Worksheet
      ws = wb.worksheets[0]
      #ws.append(new_row_data) 
      # Add row in specific index
      for idx, val in enumerate(row_list):
        ##ws.cell(row=numero_cabecera+1, column=idx+1, value=val)
        ws.cell(row=numero_cabecera+1, column=idx+1).value = ws_obj.cell(row=index+numero_cabecera-3, column=idx+1).value
        ws.cell(row=numero_cabecera+1, column=idx+1).comment =ws_obj.cell(row=index+numero_cabecera-3, column=idx+1).comment

    
      wb.save(file_name)
    except:
      print("ERROR: Revisa el nombre del fichero plantilla: "+template)
      break
  except:
    print("ERROR: Revisa la ruta objetivo: "+path)
    break

  
 

In [ ]:

###
## Process of tranlating notes to students folder
###
for index, row in df.iloc[numero_cabecera-2:].iterrows():
  # Get name to create the folder and contact mail
  name = row.values[0]  
  email = row.values[1].strip()
  # Creation of students folder
  os.chdir(path)
  os.chdir(name)
  # Transformar from numpy.ndarray to excel values
  row_list = row.values.tolist()
  # file_name = name+filenombre_excel_notas
  #@markdown ## Compartir las carpetas con los alumnos
  #@markdown Se notificará al usuario vía mail
  listed = drive.ListFile({'q': "title contains '"+name+"'"}).GetList()
  # Get las modified version
  file = listed[0]
  print("Compartiendo notas con "+email)
  try:
    file.InsertPermission({'type': 'user','value': email,'role': 'writer'})
  except:
    print("An exception occurred with email",email)
